In [1]:
# Packages laden
import numpy as np
import pandas as pd
import math
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report,confusion_matrix,auc, log_loss 
from pandas import DataFrame
from keras.backend import binary_crossentropy
import tensorflow as tf
from pandas import concat

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Laden der Daten
DATA_FILE="Dropbox/Masterarbeit/Masterarbeit/Datensätze/Geburtenrate/Datensatz.xlsx"
df = pd.read_excel(DATA_FILE)
df=pd.read_excel(DATA_FILE, header=0, parse_dates=[0], index_col='Jahr-Monat',squeeze=True)

In [3]:
# Ein array mit allen Einträgen --> zur besseren Weiterverarbeitung
ZR=df.values

In [4]:
# Skalierung [-1,1]
def scaling(dataset,l_bound=-1, h_bound=1):
    dataset = dataset.reshape(len(dataset), 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(dataset)
    scaled_dataset = scaler.transform(dataset)
    return scaled_dataset

# Umkehrung der Skalierung
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [5]:
ZR_N=scaling(ZR)

# Anteil der Datensätze am Gesamtdatensatz
ZR_100=ZR
ZR_46=ZR[-400:]

# Unterteilung der Teildatensätze in Trainings-und Testdaten
ZR_100_TR, ZR_100_TE= ZR_100[0:605], ZR_100[605:len(ZR_100)]
ZR_46_TR, ZR_46_TE= ZR_46[0:250], ZR_46[250:len(ZR_46)]

# Anteil der Datensätze am Gesamtdatensatz: skalierte Daten
ZR_N100=ZR_N
ZR_N46=ZR_N[-400:]

# Unterteilung der Teildatensätze in Trainings-und Testdaten
ZR_N100_TR, ZR_N100_TE= ZR_N100[0:605], ZR_N100[605:len(ZR_N100)]
ZR_N46_TR, ZR_N46_TE= ZR_N46[0:250], ZR_N46[250:len(ZR_N46)]

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
# Funktion zur Erzeugung von y zur Trendvorhersage/Klassifikation t+1: y=(-1,0,1)
def create_Y_trend(dataset):
    dataY=[0]*(len(dataset)-1)
    for i in range(len(dataset)-1):
        a = dataset[i]-dataset[i+1]
        if a<0:
            dataY[i]=1
        elif a>0:
            dataY[i]=-1
        else:
            dataY[i]=0
    return np.array(dataY)

In [7]:
# Funktion zur Weiterentwicklung von Trendergebnissen y(t+1) zu y(t+look_forward) mit look_forward>1
def create_Y_multitrend(dataset,look_forward):
    dataY= create_Y_trend(dataset)
    length=len(dataY)-look_forward+1
    dataY_m=[0]*length 
    dataY_m=np.array(dataY_m)
    for i in range(len(dataY_m)):
        for j in range(look_forward):
            dataY_m[i]= dataY_m[i]+dataY[i+j]
        if dataY_m[i]<0:
            dataY_m[i]=-1
        elif dataY_m[i]>0:
            dataY_m[i]=1
        else:
            dataY_m[i]=0
    return np.array(dataY_m)

In [8]:
# Funktion zur Erzeugung von x (Input) mit look_back (Anzahl der verwendeten Vergangenheitswerte)
def X_to_supervised_T(dataset, look_back=1, look_forward=1):
    df = DataFrame(dataset)
    columns = [df.shift(look_forward-1+i) for i in range(1, look_back+1)]
    df = concat(columns, axis=1)
    df = df.drop(0)
    DF=df[::-1]
    df=DF[:len(df)-(look_back+look_forward-2)]
    DF=df[::-1]
    return DF

In [30]:
# Festlegung: look_back, look_forward + Anwendung der Funktion
# Inputneuronen: 12
# Vorhersagehorizont: t+60
look_back=12
look_forward=60

# Erzeugen von y 

# Dateninput 100%
# Skalierte Daten
trainY_N100=create_Y_multitrend(ZR_N100_TR, look_forward)
testY_N100=create_Y_multitrend(ZR_N100_TE,look_forward)

# Original Daten
trainY_100=create_Y_multitrend(ZR_100_TR,look_forward)
testY_100=create_Y_multitrend(ZR_100_TE,look_forward)

# Dateninput 46%
# Skalierte Daten
trainY_N46=create_Y_multitrend(ZR_N46_TR,look_forward)
testY_N46=create_Y_multitrend(ZR_N46_TE,look_forward)

# Original Daten
trainY_46=create_Y_multitrend(ZR_46_TR,look_forward)
testY_46=create_Y_multitrend(ZR_46_TE,look_forward)

In [31]:
# Erzeugen von X 

# Dateninput: 100%
# Skalierte Daten
trainX_N100=X_to_supervised_T(ZR_N100_TR,look_back,look_forward)
testX_N100=X_to_supervised_T(ZR_N100_TE,look_back,look_forward)

#Originaldaten
trainX_100=X_to_supervised_T(ZR_100_TR,look_back,look_forward)
testX_100=X_to_supervised_T(ZR_100_TE,look_back,look_forward)

# Dateninput: 46%
# Skalierte Daten
trainX_N46=X_to_supervised_T(ZR_N46_TR,look_back,look_forward)
testX_N46=X_to_supervised_T(ZR_N46_TE,look_back,look_forward)

# Originaldaten
trainX_46=X_to_supervised_T(ZR_46_TR,look_back,look_forward)
testX_46=X_to_supervised_T(ZR_46_TE,look_back,look_forward)

In [32]:
# Zuschneiden der Datensätze --> gleiche Länge von X und y

# Dateninput: 100%
# Skalierte Daten
# Trainingsdaten
trainY_N100=trainY_N100.tolist()
trainY_N100=trainY_N100[look_back-1:]
trainY_N100=np.array(trainY_N100)
print(trainX_N100.shape, trainY_N100.shape)
# Testdaten
testY_N100=testY_N100.tolist()
testY_N100=testY_N100[look_back-1:]
testY_N100=np.array(testY_N100)
print(testX_N100.shape, testY_N100.shape)

# Originaldaten
# Trainingsdaten
trainY_100=trainY_100.tolist()
trainY_100=trainY_100[look_back-1:]
trainY_100=np.array(trainY_100)
print(trainX_100.shape, trainY_100.shape)
# Testdaten
testY_100=testY_100.tolist()
testY_100=testY_100[look_back-1:]
testY_100=np.array(testY_100)
print(testX_100.shape, testY_100.shape)

(534, 12) (534,)
(187, 12) (187,)
(534, 12) (534,)
(187, 12) (187,)


In [33]:
# Dateninput: 46%
# Skalierte Daten
# Trainingsdaten
trainY_N46=trainY_N46.tolist()
trainY_N46=trainY_N46[look_back-1:]
trainY_N46=np.array(trainY_N46)
print(trainX_N46.shape, trainY_N46.shape)
# Testdaten
testY_N46=testY_N46.tolist()
testY_N46=testY_N46[look_back-1:]
testY_N46=np.array(testY_N46)
print(testX_N46.shape, testY_N46.shape)

# Originaldaten
# Trainingsdaten
trainY_46=trainY_46.tolist()
trainY_46=trainY_46[look_back-1:]
trainY_46=np.array(trainY_46)
print(trainX_46.shape, trainY_46.shape)
# Testdaten
testY_46=testY_46.tolist()
testY_46=testY_46[look_back-1:]
testY_46=np.array(testY_46)
print(testX_46.shape, testY_46.shape)

(179, 12) (179,)
(79, 12) (79,)
(179, 12) (179,)
(79, 12) (79,)


In [34]:
# Funktion: X,y trennen, Form von X anpassen, Modell zusammenbauen, Modell anpassen
def fit_lstm(X, y , batch_size, blocks, epochen):
    X=array(X)
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(blocks, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs=epochen, batch_size=1)
    return model

In [35]:
# Replizierbarkeit gewährleisten
np.random.seed(0)

In [36]:
# Modell anpassen: 100%
lstm_model_100 = fit_lstm(trainX_N100, trainY_N100, batch_size=1, blocks=4, epochen=100)

Epoch 1/100
534/534 [==============================] - 4s 7ms/step - loss: -8.7556 - acc: 0.1873
Epoch 2/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 3/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 4/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1558 - acc: 0.1873
Epoch 5/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 6/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 7/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 8/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 9/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 10/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 11/100
534/534 [=======

534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 85/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 86/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 87/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 88/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 89/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 90/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 91/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 92/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 93/100
534/534 [==============================] - 2s 4ms/step - loss: -9.1758 - acc: 0.1873
Epoch 94/100
534/534 [===========

In [37]:
# Funktion zur Vorhersage
def forecast_lstm(model, batch_size, X):
    X=array(X)
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [38]:
# Validierung des Modells anhand der Testdaten
predictions_100 = list()
for i in range(len(testY_N100)):
    # schrittweise Vorhersage
    X=testX_N100
    yhat = forecast_lstm(lstm_model_100, 1, X.iloc[i,:])
    # Vorhersage speichern
    predictions_100.append(yhat)

In [39]:
# Umwandeln der Werte in Trendzahlen [-1,0,1]
def classification(dataset):
    for i in range(len(dataset)):
        if dataset[i]>= 0.05:
            dataset[i]=1
        elif dataset[i]<= -0.05:
            dataset[i]=-1
        else:
            dataset[i]=0
    return dataset

In [40]:
testPredict_100=classification(predictions_100)

In [41]:
accuracy_score(testY_N100, testPredict_100)

0.24598930481283424

In [25]:
# Replizierbarkeit gewährleisten
np.random.seed(1)

In [26]:
# Modell anpassen: 46%
lstm_model_46 = fit_lstm(trainX_N46, trainY_N46, batch_size=1, blocks=4, epochen=30)

Epoch 1/2
179/179 [==============================] - 2s 11ms/step - loss: -12.9956 - acc: 0.0615
Epoch 2/2
179/179 [==============================] - 1s 4ms/step - loss: -13.5068 - acc: 0.0615


In [27]:
# Validierung des Modells anhand der Testdaten
predictions_46 = list()
for i in range(len(testY_N46)):
    # schrittweise Vorhersage
    X=testX_N46
    yhat = forecast_lstm(lstm_model_46, 1, X.iloc[i,:])
    # Vorhersage speichern
    predictions_46.append(yhat)

In [28]:
testPredict=classification(predictions_46)

In [29]:
accuracy_score(testY_N46, testPredict)

0.0